# This file should be ran Locally!

### Snapdragon X Elite CRD
I recommend the following be run on your personal machine but you can do this on the Colab too however the last step takes a long time to complete.

In [ ]:
# This installs Python package
!pip install qai-hub-models[yolov8-det-quantized]
# sets up AI Hub enviroment 
!qai-hub configure --api_token <YOUR_API_TOKEN>
# Converts the model to be ran on Snapdragon X Elite CRD
!python -m qai_hub_models.models.yolov8_det_quantized.export --device="Snapdragon X Elite CRD" --ckpt-name /weights/best.pt


In [ ]:
import tensorflow as tf

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="PATH_TO_FILE_FROM_AIHUB")
interpreter.allocate_tensors()

# Get output tensor details
output_details = interpreter.get_output_details()

q_scales = []
q_zero_points = []

for output in output_details:
    params = output['quantization_parameters']

    # Check if the tensor is quantized
    if output['quantization_parameters']['quantized_dimension'] == 0:  # Per-tensor quantization
        scale = params['scales'][0] if params['scales'].size > 0 else 1.0
        zero_point = params['zero_points'][0] if params['zero_points'].size > 0 else 0.0
    else:
        # Handle per-channel quantization (unlikely for YOLOv8)
        scale = 1.0
        zero_point = 0.0

    q_scales.append(scale)
    q_zero_points.append(float(zero_point))

# Format the constants
print(f'YOLOv8,q-offsets=<{", ".join(map(str, q_zero_points))}>, q-scales=<{", ".join(map(str, q_scales))}>')